### 1. Configuraçoes Iniciais

Imports

In [196]:
import requests
import pandas as pd
import numpy as np
import os

Importando dados do funds explorer

In [197]:
url = 'https://www.fundsexplorer.com.br/ranking'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36'' (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    df = pd.read_html(response.content, encoding='utf-8')[0]

### 2. Limpeza e Formatação de Dados

Dados categóricos

In [198]:
categorical_columns = ['Código do fundo','Setor'] #cria uma variavel para as colunas categoricas
idx = df[df['Setor'].isna()].index #mostra quais linhas possuem dados NA
df.drop(idx, inplace=True) #apaga estas linhas
df[categorical_columns] = df[categorical_columns].astype('category') # define as colunas ocmo categoricas

Dados Numéricos

In [199]:
col_floats = list(df.iloc[:,2:-1].columns) #Seleciona da terceira até a penultima linha 
df[col_floats] = df[col_floats].fillna(value=0) #substitui os dados NA por zero
df[col_floats] #valida se existe ainda dados nulos
df[col_floats] = df[col_floats].applymap(lambda x: str(x).replace('R$', '').replace('.0','').replace('.','').replace('%','').replace(',','.')) #limpa os dados removendo caracteres inuteis
df[col_floats] = df[col_floats].astype('float') #define as colunas como float

Dados de P/VPA tem atributos infinitos e está em uma escala diferente

In [205]:
df = df.replace([np.inf, -np.inf], np.nan)  # Converter infinitos para NaN
df = df.dropna() #apaga os dados NA
df['P/VPA'] = df['P/VPA']/100 #convertendo P/VPA para

### 3. Analises (DF)

In [206]:
indicadores = ['Código do fundo','Setor', 'Preço Atual','DY (12M) Acumulado', 'Vacância Física', 'Vacância Financeira', 'P/VPA', 'Quantidade Ativos', 'Liquidez Diária']
indicadoresf = ['DY (12M) Acumulado', 'Vacância Física', 'Vacância Financeira', 'P/VPA', 'Quantidade Ativos', 'Liquidez Diária']

df_aux = df[indicadores] #filtra o df pelos  cabeçalhos
display(df_aux.head())

,Código do fundo,Setor,Preço Atual,DY (12M) Acumulado,Vacância Física,Vacância Financeira,P/VPA,Quantidade Ativos,Liquidez Diária
0,FIVN11,Shoppings,3.09,0.0,56.00,0.0,0.0040,1,49265.0
1,BZLI11,Títulos e Val. Mob.,17.11,0.0,0.00,0.0,0.0160,0,3.0
2,XTED11,Lajes Corporativas,8.82,0.0,0.00,100.0,0.0067,1,2223.0
3,ALMI11,Lajes Corporativas,765.00,0.0,64.05,0.0,0.0038,1,20.0
4,PABY11,Híbrido,12.09,0.0,0.00,0.0,-0.0069,1,41.0


#### Criando uma função com uma estratégia para oportunidades do mercado

In [207]:
def oportunidade_media_setor(df, setor, label_setor='Setor'):
    
    media_setor = df.groupby('Setor')[indicadoresf].agg(['mean']) #calcula a média dos indicadores do setor escolhido
    
    df_setor = df[df[label_setor].isin([setor])]
    
    filter_ = \
            (df_setor['Quantidade Ativos'] > 1) &\
            (df_setor['Liquidez Diária'] > 1) &\
            (df_setor['P/VPA'] < 2.0) &\
            (df_setor['DY (12M) Acumulado'] > media_setor.loc[setor, ('DY (12M) Acumulado','mean')]) 
            
    print('# média do setor Yield: {}'.format(media_setor.loc[setor, ('DY (12M) Acumulado','mean')]))
    print('# média do setor p/VPA: {}'.format(media_setor.loc[setor, ('P/VPA','mean')]))
    print('# média do setor Ativos: {}'.format(media_setor.loc[setor, ('Quantidade Ativos','mean')]))
    
    return df_setor[filter_]

In [208]:
dataframe = df_aux
setorF = 'Híbrido'

oportunidade = oportunidade_media_setor(dataframe, setorF)
oportunidade.sort_values('DY (12M) Acumulado', ascending=False, inplace=True)
oportunidade

# média do setor Yield: 9.32433962264151
# média do setor p/VPA: 0.0074264150943396225
# média do setor Ativos: 3.4150943396226414


,Código do fundo,Setor,Preço Atual,DY (12M) Acumulado,Vacância Física,Vacância Financeira,P/VPA,Quantidade Ativos,Liquidez Diária
261,BREV11,Híbrido,90.04,18.97,0.00,0.0,0.0077,12,31.0
94,MFII11,Híbrido,92.17,13.94,0.00,0.0,0.0089,9,2842.0
27,ARCT11,Híbrido,87.13,13.92,0.00,0.0,0.0087,4,6496.0
174,RZTR11,Híbrido,87.25,13.39,0.00,0.0,0.0091,10,25967.0
172,NEWL11,Híbrido,100.99,12.28,0.00,0.0,0.0084,2,753.0
35,RELG11,Híbrido,53.20,12.28,10.40,10.4,0.0043,3,3725.0
139,SARE11,Híbrido,60.30,11.42,6.20,0.0,0.0065,3,23052.0
272,RECT11,Híbrido,55.18,11.11,15.10,0.0,0.0060,8,13320.0
200,GALG11,Híbrido,8.89,10.69,0.00,0.0,0.0096,6,65542.0
264,FATN11,Híbrido,94.40,10.68,4.30,0.0,0.0095,7,224.0


### 4. Analises (FIIs Pessoais)

#### 4.1 Importacao de Tickets via CSV

In [204]:
os.chdir("C:/Users/luis-/projetos/Python/analise-acoes/") #selecionando o diretorio do arquivo CSV a ser lido
tickets = pd.read_csv("./tickets.csv", sep = ',') #importando dados de um csv utilizando o pandas
tickets = tickets['Ativo'].tolist() #convertendo o data frame para lista
filtro = df.iloc[:, 0].isin(tickets) #filtando os dados utilizando as acoes do arquivo csv
df_filtro = df.loc[filtro]
display(df_filtro)

,Código do fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,DY (3M) Média,...,Patrimônio Líq.,VPA,P/VPA,DY Patrimonial,Variação Patrimonial,Rentab. Patr. no Período,Rentab. Patr. Acumulada,Vacância Física,Vacância Financeira,Quantidade Ativos
57,ALZR11,Logística,111.99,12006.0,0.82,0.72,2.18,4.21,12.41,0.73,...,7.403375e+08,106.94,1.05,0.68,-0.03,0.65,7.26,0.00,0.00,15
70,KNRI11,Híbrido,153.49,23916.0,0.95,0.65,2.03,4.04,7.80,0.68,...,3.869520e+09,160.18,0.96,0.59,0.14,0.73,2.56,2.00,6.72,20
86,PVBI11,Lajes Corporativas,96.21,25668.0,0.61,0.64,2.03,3.99,7.55,0.68,...,1.240685e+09,102.18,0.94,0.60,-1.42,-0.83,-0.83,0.10,0.00,4
105,HGRE11,Lajes Corporativas,127.18,19548.0,0.78,0.62,2.02,4.20,7.71,0.67,...,1.847756e+09,156.35,0.81,0.50,-0.33,0.17,1.60,27.13,29.17,21
107,HGRU11,Híbrido,124.17,25568.0,0.85,0.69,2.17,5.16,9.77,0.72,...,2.276574e+09,123.68,1.00,0.69,0.06,0.75,2.80,0.00,0.00,17
135,HGCR11,Títulos e Val. Mob.,103.49,26264.0,1.20,1.16,3.53,7.04,14.06,1.18,...,1.574428e+09,102.12,1.01,1.18,0.40,1.58,5.10,0.00,0.00,0
137,MXRF11,Híbrido,10.72,820605.0,0.12,1.13,3.47,6.35,12.74,1.16,...,2.284498e+09,10.11,1.06,0.00,0.00,0.00,0.00,0.00,0.00,0
165,XPLG11,Logística,105.19,25809.0,0.74,0.74,2.32,4.62,8.86,0.77,...,3.824890e+08,113.74,0.92,0.65,0.28,0.94,2.11,8.80,1.20,13
204,HGLG11,Logística,162.90,49588.0,1.10,0.68,2.05,4.74,10.03,0.68,...,3.579871e+08,152.58,1.07,0.72,0.05,0.78,1.75,6.90,6.50,17
287,RBRP11,Outros,53.50,13056.0,0.27,0.55,1.76,3.42,7.80,0.59,...,9.553410e+08,78.44,0.68,0.34,1.04,1.38,1.79,13.00,6.00,17


In [184]:
df_auxp = df_filtro[indicadores] #filtra o df pelos  cabeçalhos
filter_ = (df_aux['P/VPA'] < 1.01) & (df_aux['P/VPA'] > 0.85)

df_auxp = df_auxp[(filter_)].sort_values(['P/VPA'])
display(df_auxp)

C:\Users\luis-\AppData\Local\Temp\ipykernel_18588\98679463.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_auxp = df_auxp[(filter_)].sort_values(['P/VPA'])


,Código do fundo,Setor,Preço Atual,DY (12M) Acumulado,Vacância Física,Vacância Financeira,P/VPA,Quantidade Ativos,Liquidez Diária
308,BRCO11,Logística,"R$ 108,50","7,85%","0,00%","0,00%",89.0,10,29879.0
165,XPLG11,Logística,"R$ 105,19","8,86%","8,80%","1,20%",92.0,13,25809.0
86,PVBI11,Lajes Corporativas,"R$ 96,21","7,55%","0,10%",NaN,94.0,4,25668.0
70,KNRI11,Híbrido,"R$ 153,49","7,80%","2,00%","6,72%",96.0,20,23916.0
323,BTLG11,Logística,"R$ 97,99","9,10%",NaN,"2,00%",99.0,16,37684.0
107,HGRU11,Híbrido,"R$ 124,17","9,77%","0,00%","0,00%",100.0,17,25568.0
